In [70]:
import numpy as np
import pandas as pd


path = './data/'
output = './output/'


In [60]:
# start with an effort to figure out if players are returning the next year
# sometimes their name gets inexplicably reversed, in fact most are reversed in 2018
# need column that answers question (did return?)
# merge in player stats

# ONLY NEED TO RUN ONE TIME
# years = list(range(2010, 2020))
# years = list(map(str, years))
# years.reverse()

# for y in years:
#     player_ext = "Players_{}.csv".format(y)
#     players = pd.read_csv(path+player_ext)
#     if years.index(y) != 0:
#         players = pd.merge(players, ny_players, how='left', on=['TeamID','TeamID'])
      
#     players['AltName'] = players['PlayerName'].str.replace('_',' ')
#     players['AltName'] = ['_'.join(s.split()[::-1]) for s in players['AltName']]
    
# #     players that will play the next year (since I'm going backward)
#     ny_players = players.groupby(['TeamID'])['PlayerName'].apply(list).reset_index(name='NY_Players')
#     ny_players = ny_players[['TeamID','NY_Players']]
#     if years.index(y) != 0:
#         players['NY_Players'] = players['NY_Players'].apply(lambda d: d if isinstance(d, list) else [])
#         players['WillReturn'] = players.apply(lambda x: x['PlayerName'] in x['NY_Players'], axis=1)
#         players['WillReturn2'] = players.apply(lambda x: x['AltName'] in x['NY_Players'], axis=1)
#         players['Returning'] = np.where((players['WillReturn']|players['WillReturn2']), True, False)
#         players = players[['PlayerID','Season','TeamID','PlayerName','AltName','Returning']]
#         players = players.loc[players['PlayerName']!='TEAM']
#         players.to_csv(path+'New_Players_{}.csv'.format(y),index=None)

In [15]:
list(events)

['EventID',
 'Season',
 'DayNum',
 'WTeamID',
 'LTeamID',
 'WPoints',
 'LPoints',
 'ElapsedSeconds',
 'EventTeamID',
 'EventPlayerID',
 'EventType']

In [18]:
# make sure score keeping is relatively consistent
# events.groupby(['EventType'])['EventID'].count()

# 2018 numbers
#made2_dunk     18975
# made2_jump     64606
# made2_lay     110309
# made2_tip       4198

# 2010 numbers
# made2_dunk     16065
# made2_jump     69599
# made2_lay      97364
# made2_tip       4416

# seems okay, scoring seems to have gone up over time, or there are more games

EventType
assist        135116
block          36076
foul_pers     196038
foul_tech       1486
made1_free    151741
made2_dunk     16065
made2_jump     69599
made2_lay      97364
made2_tip       4416
made3_jump     65164
miss1_free     59367
miss2_dunk      1829
miss2_jump    128469
miss2_lay      72835
miss2_tip       2447
miss3_jump    125362
reb_dead       32061
reb_def       247105
reb_off       119795
steal          69508
sub_in        325212
sub_out       326376
timeout        36260
timeout_tv     38518
turnover      144421
Name: EventID, dtype: int64

In [19]:
# only use events we care about
# print(list(events.EventType.unique()))

dont_care = ['foul_pers','foul_tech','timeout_tv', 'reb_dead', 'timeout','sub_in', 'sub_out']
# i don't think i'm going to tally missed shots
missed = ['miss2_jump','miss1_free','miss3_jump', 'miss2_lay',  'miss2_dunk', 'miss2_tip']

event_types = ['assist', 'made3_jump', 'turnover',  'reb_off', 'made2_dunk','made2_jump', 'steal', 'made1_free', 'block','reb_def','made2_lay', 'made2_tip']



2502630
1156370


In [63]:
def filter_events(df):
    df = df.loc[df.EventType.isin(event_types)]
    return df



In [64]:
# categories I'm interested in year over yeat
# aggregate event types
rebounds = ['reb_off','reb_def']
# close range 2pts
cr_2pt = ['made2_lay', 'made2_tip', 'made2_dunk']
# mid range 2pts
mr_2pt = ['made2_jump']
# 3 points
lr = ['made3_jump']
# assists, blocks, steals, fts
# 8 total categories

In [65]:
def aggregate_events(df):
    
    df.loc[df.EventType.isin(rebounds), 'EventType'] = 'rebound'
    df.loc[df.EventType.isin(cr_2pt), 'EventType'] = 'cr_2pt'
    
    return df



In [66]:
# can group events by player id

def player_level_events(df):
    df = df.groupby(['EventPlayerID','EventType'])['EventID'].count().unstack(-1).reset_index().fillna(0)
    df = df.rename(columns={'EventPlayerID':'PlayerID'})
    return df


In [67]:
teams_ext = 'Teams.csv'
teams = pd.read_csv(path+teams_ext)


In [71]:
years = list(range(2010, 2019))
years = list(map(str, years))

for y in years:
    player_ext = 'New_Players_{}.csv'.format(y)
    event_ext = 'Events_{}.csv'.format(y)
    events = pd.read_csv(path+event_ext)
    players = pd.read_csv(path+player_ext)
    
    events = filter_events(events)
    events = aggregate_events(events)
    events = player_level_events(events)
    
    players = pd.merge(players, events, how='left', on=['PlayerID','PlayerID'])
    players = players.loc[players.PlayerName != 'TEAM']
    players = players.fillna(0)
    #player level season stats, and if they return
    players.to_csv(output+'PL_Season_Stats_{}.csv'.format(y),index=None)
    




In [72]:
print(players.head())

   PlayerID  Season  TeamID       PlayerName          AltName  Returning  \
0    648095    2018    1101       COBB_CHASE       CHASE_COBB       True   
1    648096    2018    1101   FARQHAR_HAYDEN   HAYDEN_FARQHAR      False   
2    648097    2018    1101  FARQUHAR_HAYDEN  HAYDEN_FARQUHAR       True   
3    648098    2018    1101     FOSTER_TEVIN     TEVIN_FOSTER      False   
4    648099    2018    1101  FRANKLIN_JAYLEN  JAYLEN_FRANKLIN       True   

   assist  block  cr_2pt  made1_free  made2_jump  made3_jump  rebound  steal  \
0     0.0    0.0     0.0         0.0         0.0         0.0      0.0    0.0   
1     0.0    1.0     0.0         0.0         0.0         1.0      1.0    0.0   
2     9.0    6.0    14.0         4.0         2.0        13.0     51.0   10.0   
3    89.0    4.0    43.0        94.0        18.0        43.0     82.0   38.0   
4   117.0    5.0    85.0        48.0        31.0         4.0    114.0   45.0   

   turnover  
0       0.0  
1       2.0  
2      11.0  
3     

In [73]:
# team level returning
# for y in years:
    

assist  block  cr_2pt
Season TeamID Returning                       
2018   1101   False       127.0    9.0    74.0
              True        248.0   89.0   258.0
       1102   False       246.0   36.0   133.0
              True        156.0   40.0   130.0
       1103   False       216.0   13.0   166.0
              True        185.0   61.0   185.0
       1104   False       194.0   17.0   163.0
              True        264.0  175.0   318.0
       1105   False       199.0   17.0   163.0
              True        141.0   32.0   160.0
       1106   False        77.0   20.0    54.0
              True        236.0  110.0   283.0
       1107   False       311.0   70.0   337.0
              True         77.0   21.0    63.0
       1108   False       223.0   41.0   151.0
              True        107.0   35.0   149.0
       1110   False        76.0   21.0    43.0
              True        291.0   77.0   323.0
       1111   False        63.0   30.0   114.0
              True        304.0   73.0   265.0
       1112   False       412.0  132.0   431.0
              True        118.0   26.0    45.0
       1113   False       287.0   16.0   137.0
              True        171.0   98.0   301.0
       1114   False       204.0   55.0   262.0
              True        142.0   32.0   111.0
       1115   False       114.0   88.0   150.0
              True        307.0   56.0   176.0
       1116   False       452.0   61.0   308.0
              True         49.0  102.0   184.0
...                         ...    ...     ...
       1450   False       287.0   26.0   182.0
              True        163.0   45.0   156.0
       1451   False       180.0   10.0    37.0
              True        192.0   98.0   275.0
       1452   False       389.0   30.0   204.0
              True        172.0  162.0   215.0
       1453   False       217.0   42.0   167.0
              True        185.0   45.0   112.0
       1454   False       346.0  104.0   324.0
              True         61.0   14.0    64.0
       1455   False       584.0  123.0   425.0
              True         26.0    4.0    35.0
       1456   False       285.0   10.0   110.0
              True        206.0   98.0   261.0
       1457   False       148.0   65.0   115.0
              True        240.0   68.0   214.0
       1458   False        30.0    9.0    18.0
              True        381.0  108.0   385.0
       1459   False        46.0   15.0    35.0
              True        379.0   63.0   243.0
       1460   False       130.0   21.0   124.0
              True        302.0   54.0   271.0
       1461   False       323.0  103.0   248.0
              True        176.0   20.0   134.0
       1462   False       269.0   64.0   303.0
              True        310.0   45.0   249.0
       1463   False        38.0    7.0    19.0
              True        438.0   80.0   392.0
       1464   False       329.0  100.0   289.0
              True         67.0   12.0   121.0

[702 rows x 3 columns]